In [ ]:
from db import DB
from models.pytorch.us_gaap_alignment.ingest import load_ofss_schema, insert_ofss_data

db = DB()

# Path to OFSS schema file
schema_file = '../shared/open_financial_statement_schema.json'

# Load the schema from the JSON file
schema_data = load_ofss_schema(schema_file)

# Insert the data into the database
insert_ofss_data(db, schema_data)